# Challenge Nubimetrics - Escobar Aldo

In [1]:
import pandas as pd
import urllib.request
import re
from bs4 import BeautifulSoup

## Funcion mapper que asigna la marca correspondiente a la row segun el titulo. 

In [2]:
def mapper(list_values):
    for key, values in dict_brands.items():
        if any(value in list_values for value in values):
            return key
    return 'otros'

## Función para normalizar strings (lo hace minúscula y realiza ciertas correcciones como acentos o caracteres especiales mediante una expresión regular)

In [3]:
def normalize(string):
    return re.sub("[^0-9a-zA-Z]+", " ", string.lower())

## Data Scraping

Realizo un scrapping de datos de gsmarena, la cual contiene un listado de marcas de celulares. El data set contiene otras marcas, las cuales quizás sean imitaciones o marcas nacionales, por lo que decidí agregarlas a mano en una lista, con un previo análisis sobre el filtrado de datos que realicé, para ir sacando estas marcas adicionales del label "otros". Se podría adicionar datasets de marcas de celulares extranjeros, teniendo en cuenta la posibilidad de venta de estos en el país. De los nacionales e imitaciones no pude encontrar dataset, por lo que decidí agregarlos en una lista, como mencioné anteriormente.


In [4]:
brands_gsmarena = []
brands_page = 'https://www.gsmarena.com/makers.php3'
webpage = urllib.request.urlopen(brands_page)
soup = BeautifulSoup(webpage, 'html.parser')
table = soup.find_all('table')[0]
table_a = table.find_all('a')
for a in table_a:
    temp = a['href'].split('-')[0]
    brands_gsmarena.append(temp)
brands_gsmarena

['acer',
 'alcatel',
 'allview',
 'amazon',
 'amoi',
 'apple',
 'archos',
 'asus',
 'at&t',
 'benefon',
 'benq',
 'benq_siemens',
 'bird',
 'blackberry',
 'blackview',
 'blu',
 'bosch',
 'bq',
 'casio',
 'cat',
 'celkon',
 'chea',
 'coolpad',
 'dell',
 'emporia',
 'energizer',
 'ericsson',
 'eten',
 'fujitsu_siemens',
 'garmin_asus',
 'gigabyte',
 'gionee',
 'google',
 'haier',
 'honor',
 'hp',
 'htc',
 'huawei',
 'i_mate',
 'i_mobile',
 'icemobile',
 'infinix',
 'innostream',
 'inq',
 'intex',
 'jolla',
 'karbonn',
 'kyocera',
 'lava',
 'leeco',
 'lenovo',
 'lg',
 'maxon',
 'maxwest',
 'meizu',
 'micromax',
 'microsoft',
 'mitac',
 'mitsubishi',
 'modu',
 'motorola',
 'mwg',
 'nec',
 'neonode',
 'niu',
 'nokia',
 'nvidia',
 'o2',
 'oneplus',
 'oppo',
 'orange',
 'palm',
 'panasonic',
 'pantech',
 'parla',
 'philips',
 'plum',
 'posh',
 'prestigio',
 'qmobile',
 'qtek',
 'razer',
 'realme',
 'sagem',
 'samsung',
 'sendo',
 'sewon',
 'sharp',
 'siemens',
 'sonim',
 'sony',
 'sony_ericss

In [5]:
other_brands = ['bgh', 'noblex','philco','pcd','sansei','redmi','nextel','panacom','tcl','quantum','top house','doogee','kanji','kodak','pcbox','sky','neso','ken brown','akua','exo','admiral','daewoo','ulefone','figo','datsun','dtc','lumee','woo','overtech','genlus touch']

In [6]:
df=pd.read_csv("cellphoneslisting.csv", sep="")
df["title_new"]=df["title"].apply(normalize)
df.head()

,id,title,sales,unit_sales,title_new
0,1,Nokia Asha 503 Dual Sim Liberado Nuevos Gar...,2996.99,1,nokia asha 503 dual sim liberado nuevos garantia
1,2,Samsung Galaxy A5 A500 16gb Refabricado 2gb Ra...,31495.00,5,samsung galaxy a5 a500 16gb refabricado 2gb ra...
2,3,Celular Android Smartphone Liberado 4g Lte Tcl...,77380.00,20,celular android smartphone liberado 4g lte tcl...
3,4,Telefono Celular Lg G7 4g Lte Libre Nuevo Gara...,99997.00,3,telefono celular lg g7 4g lte libre nuevo gara...
4,5,Telefono Celular Lg G7 New Platinum Gray 4g Lt...,219992.00,8,telefono celular lg g7 new platinum gray 4g lt...


## Armado del diccionario filtrador y mapeo
Se arma un diccionario que contiene como claves las marcas, tanto de las scrapeadas como de la lista adicional que declaré antes. 
Como valor contiene el valor de la clave en sí misma en forma de una lista. Había ciertos errores en la escritura o algunos modelos correspondientes en la marca, por lo que decidí agregarlos en la lista de valores según la clave a la que corresponde.
El matching brand-model podría mejorarse como también aquellas palabras que están mal escritas, ver _Consideraciones_.

Luego, invoqué la función definida anteriormente en un map, la cual se aplica para cada valor de la dataframe, asignando su marca correspondiente. Esto se añade a una nueva columna brand de la dataframe.

In [7]:
dict_brands = dict(zip(sorted(brands_gsmarena+other_brands), [[b] for b in sorted(brands_gsmarena+other_brands)]))
dict_brands.update({"apple":["apple", "iphone"]})
dict_brands.update({"motorola":["motorola", "moto", "z2 play"]})
dict_brands.update({"samsung":["samsung", "sansung", "galaxy", "samgung", "samsumg", "samung", "grand prime", "j7 prime", "j5 prime", "j4", ]})
dict_brands.update({"xiaomi":["xiaomi", "xiomi", "xioami"]})
dict_brands.update({"blackberry":["blackberry", "blacberry"]})
dict_brands.update({"oneplus":["oneplus", "one plus"]})
dict_brands.update({"huawey":["huawey", "huawie"]})
dict_brands.update({"alcatel":["alcatel", "alkatel"]})
dict_brands.update({"blu":["blu", "diva flex 2"]})
dict_brands.update({"lg":["lg", "k10"]})
dict_brands['ipro'] =['ipro', 'i-pro']

In [8]:
df['brand'] = df.title_new.map(mapper)

## GroupBy, generación del output y chequeo
Agrupo por marca, lo contalbilizo y ordeno esta agrupación en orden descendente tomando por referencia las unidades vendidas. Hago una proyección/filtrado para que solo me muestre las ventas en pesos y unidades.
En la siguiente linea reseteo el indice, el cual se agrega como columna en la dataframe agrupada.
Finalmente, exporto la dataframe como csv.
La última linea corresponde a un chequeo que hacía sobre el label "otros", para ir viendo los datos, identificando nuevas marcas y algunas incoherencias, por ejemplo para el caso de "liberar icloud", "mayorista de celular", "vidriorepuesto s7 edge", "celulares por mayor", entre otros.
Estos, pueden ser borrados luego de leer los datos de "cellphoneslisting.csv", pero los dejo en este caso para que pueda verse lo que dije anteriormente.

In [9]:
df.groupby('brand').sum().sort_values(by=['unit_sales'], ascending=False)[["sales", "unit_sales"]]

,sales,unit_sales
brand,,
samsung,88362251.59,8967
motorola,55379203.59,5624
lg,35185172.47,5490
redmi,29019898.24,2857
xiaomi,31967216.30,2241
huawei,21522076.32,1552
blu,4573255.79,1374
nokia,1632086.18,623
apple,14745584.21,545


In [10]:
output=df.groupby('brand').sum().sort_values(by=['unit_sales'], ascending=False)[["sales", "unit_sales"]].reset_index()
output

,brand,sales,unit_sales
0,samsung,88362251.59,8967
1,motorola,55379203.59,5624
2,lg,35185172.47,5490
3,redmi,29019898.24,2857
4,xiaomi,31967216.30,2241
5,huawei,21522076.32,1552
6,blu,4573255.79,1374
7,nokia,1632086.18,623
8,apple,14745584.21,545
9,otros,877484.27,508


In [11]:
df[df.brand=="otros"].sort_values(by=['unit_sales'], ascending=False)[["id", "title", "sales", "unit_sales"]]

,id,title,sales,unit_sales
950,951,Celular Libre Barato Super Fino Tipo Tarjeta B...,77870.00,130
99,100,Celular Liberado 5'' Quad Core 8mp + 2mp 1gb ...,271102.00,98
2565,2566,Celular Por Mayor (por Favor Leer Bien La Desc...,47.43,47
3709,3710,Liberar Icloud,5215.00,35
2527,2528,Mayorista De Celular,34.19,34
100,101,Celular Liberado 5'' Quad Core Lte 4g 8mpx 1gb...,86971.00,29
1151,1152,Celular Smartphone 4g Lte Quad Core 5 Pul Dual...,39990.00,10
288,289,Celular Libre Teclas Grandes Basico Camara Radio,6291.00,9
1464,1465,Celulares Por Mayor,8.00,8
518,519,Vidriorepuesto S7 Edge,19600.00,7


<u>Consideraciones:</u>
* Para hacer que esto sea más escalable aún (ya que suele haber publicaciones en donde sólo nombran el modelo), una posible accion es scrapear modelos y vincularlos con su respectiva marca. De tal forma que exista una conexión 1 a muchos (1 brand - Muchos modelos).
* Podría usarse, si se dispone de un data set de marcas completo, un algoritmo que calcule la _Distancia Damerau-Levenshtein_.  El algoritmo se puede utilizar para crear sugerencias de corrección ortográfica, encontrando la palabra más cercana de una lista. Esto podría aplicarse luego de matchear los titles del data set con brand y de matchear modelos con brand, como mencioné en el punto anterior, ya que si lo hacemos de antemano, puede que termine vinculando un modelo con una marca, provocando un error en los datos.
* 7puentes escribió un artículo que hace referencia a este problema del matching de productos: http://www.7puentes.com/blog/2018/10/22/machine-learning-casos-de-negocios/. Podría considerarse también el uso de _TF-IDF_ para ponderar las palabras y de _Random Forest_ como clasificador.